# Introduction

The [https://badssl.com](https://badssl.com) website offers a variety of unique scenarios with TLS protocols, cipher suites, and certificates.  In this case, we are using the web site to identify the difference between certificate thumbprints and thrumbprints against each certificate SubjectPublicKeyInfo field.  This work borrows from the research identified from the [OpenSSL Notebook](./openssl.ipynb).

## Computing Environemnt

Some basic information about the environment in which the Nobeook is executing.

In [12]:
# What environment am I running in?
!echo "Kernel:" && uname -a && echo ""
!echo "MacOS Version:" && sw_vers && echo ""
!echo "OpenSSL Version:" && openssl version && echo ""
!echo "Python Version:" && python --version && echo ""

Kernel:
Darwin USWSA220038.local 23.6.0 Darwin Kernel Version 23.6.0: Wed Jul 31 20:49:39 PDT 2024; root:xnu-10063.141.1.700.5~1/RELEASE_ARM64_T6000 arm64

MacOS Version:
ProductName:		macOS
ProductVersion:		14.7
BuildVersion:		23H124

OpenSSL Version:
OpenSSL 3.3.1 4 Jun 2024 (Library: OpenSSL 3.3.1 4 Jun 2024)

Python Version:
Python 3.11.6



## Acquire Subdomains

Crawl the top-level page of [https://badssl.com](https://badssl.com) for any related sub-domains.

In [21]:
%%bash
FOLDER=data/badssl/
mkdir -p ${FOLDER}
DOMAINS=${FOLDER}/domains.txt

curl -s https://badssl.com/ \
| grep -Eo '<a href="(.*badssl\.com.*?)"' \
| awk -F '/' '{print $3}' \
| { echo "www.badssl.com"; echo "badssl.com"; cat -; } \
| grep 'badssl' \
| sort | uniq > "${DOMAINS}"

echo "Domains found: $(wc -l ${DOMAINS} | awk '{print $1}')"
echo ""
cat ${DOMAINS}


Domains found: 74

1000-sans.badssl.com
10000-sans.badssl.com
3des.badssl.com
badssl.com
captive-portal.badssl.com
cbc.badssl.com
client-cert-missing.badssl.com
client.badssl.com
dh-composite.badssl.com
dh-small-subgroup.badssl.com
dh1024.badssl.com
dh2048.badssl.com
dh480.badssl.com
dh512.badssl.com
dsdtestprovider.badssl.com
ecc256.badssl.com
ecc384.badssl.com
edellroot.badssl.com
expired.badssl.com
extended-validation.badssl.com
hsts.badssl.com
http-credit-card.badssl.com
http-dynamic-login.badssl.com
http-login.badssl.com
http-password.badssl.com
http-textarea.badssl.com
http.badssl.com
https-everywhere.badssl.com
incomplete-chain.badssl.com
invalid-expected-sct.badssl.com
lock-title.badssl.com
long-extended-subdomain-name-containing-many-letters-and-dashes.badssl.com
longextendedsubdomainnamewithoutdashesinordertotestwordwrapping.badssl.com
mitm-software.badssl.com
mixed-favicon.badssl.com
mixed-form.badssl.com
mixed-script.badssl.com
mixed.badssl.com
mozilla-intermediate.badssl.c

## Acquire Certificates by Domain

Using the domain target list, retireve the certificates presented for each domain.

> Note: Some of the subdomains offer risky TLS configurations which modern versions of `openssl` may not support.  You can expect some errors to happen due to this incompatibility.  There will be enough sample data to illustrate the differences between thumbprint types.

In [23]:
%%bash
BASE_FOLDER="data/badssl"
CERT_FOLDER="${BASE_FOLDER}/certificates"
mkdir -p "${CERT_FOLDER}"
DOMAIN_LIST="${BASE_FOLDER}/domains.txt"

for DOMAIN in $(cat ${DOMAIN_LIST}); 
    do
    # assign port 443 if not specified
    PORT=$(echo "${DOMAIN}" | awk -F ':' '{ if (NF == 1) print 443; else if (NF == 2) print $2 }')
    DOMAIN=$(echo "${DOMAIN}" | awk -F ':' '{print $1}')

    echo "Acquiring Certificate: https://${DOMAIN}:${PORT}"

    DOMAIN=$(echo "${DOMAIN}" | awk -F ':' '{print $1}')
    
    echo "" | openssl s_client -showcerts -servername "${DOMAIN}" -connect "${DOMAIN}:${PORT}" 2>/dev/null |\
        openssl x509 -outform PEM -out "${CERT_FOLDER}/${DOMAIN}_${PORT}"_signed-public-key.pem;

    openssl x509 -inform PEM -in "${CERT_FOLDER}/${DOMAIN}_${PORT}"_signed-public-key.pem \
        -outform DER -out "${CERT_FOLDER}/${DOMAIN}_${PORT}"_signed-public-key.der
    done

Acquiring Certificate: https://1000-sans.badssl.com:443
Acquiring Certificate: https://10000-sans.badssl.com:443


Could not find certificate from <stdin>
408FAAEC01000000:error:1608010C:STORE routines:ossl_store_handle_load_result:unsupported:crypto/store/store_result.c:151:
Could not open file or uri for loading certificate from data/badssl/certificates/10000-sans.badssl.com_443_signed-public-key.pem: No such file or directory


Acquiring Certificate: https://3des.badssl.com:443


Could not find certificate from <stdin>
408FAAEC01000000:error:1608010C:STORE routines:ossl_store_handle_load_result:unsupported:crypto/store/store_result.c:151:
Could not open file or uri for loading certificate from data/badssl/certificates/3des.badssl.com_443_signed-public-key.pem: No such file or directory


Acquiring Certificate: https://badssl.com:443
Acquiring Certificate: https://captive-portal.badssl.com:443
Acquiring Certificate: https://cbc.badssl.com:443
Acquiring Certificate: https://client-cert-missing.badssl.com:443
Acquiring Certificate: https://client.badssl.com:443
Acquiring Certificate: https://dh-composite.badssl.com:443
Acquiring Certificate: https://dh-small-subgroup.badssl.com:443
Acquiring Certificate: https://dh1024.badssl.com:443
Acquiring Certificate: https://dh2048.badssl.com:443
Acquiring Certificate: https://dh480.badssl.com:443
Acquiring Certificate: https://dh512.badssl.com:443
Acquiring Certificate: https://dsdtestprovider.badssl.com:443
Acquiring Certificate: https://ecc256.badssl.com:443
Acquiring Certificate: https://ecc384.badssl.com:443
Acquiring Certificate: https://edellroot.badssl.com:443
Acquiring Certificate: https://expired.badssl.com:443
Acquiring Certificate: https://extended-validation.badssl.com:443
Acquiring Certificate: https://hsts.badssl.com:

...

Acquiring Certificate: https://revoked.badssl.com:443
Acquiring Certificate: https://rsa2048.badssl.com:443
Acquiring Certificate: https://rsa4096.badssl.com:443
Acquiring Certificate: https://rsa8192.badssl.com:443
Acquiring Certificate: https://self-signed.badssl.com:443
Acquiring Certificate: https://sha1-2016.badssl.com:443
Acquiring Certificate: https://sha1-2017.badssl.com:443
Acquiring Certificate: https://sha1-intermediate.badssl.com:443
Acquiring Certificate: https://sha256.badssl.com:443
Acquiring Certificate: https://sha384.badssl.com:443
Acquiring Certificate: https://sha512.badssl.com:443
Acquiring Certificate: https://spoofed-favicon.badssl.com:443
Acquiring Certificate: https://static-rsa.badssl.com:443
Acquiring Certificate: https://subdomain.preloaded-hsts.badssl.com:443
Acquiring Certificate: https://superfish.badssl.com:443
Acquiring Certificate: https://tls-v1-0.badssl.com:1010


Could not find certificate from <stdin>
408FAAEC01000000:error:1608010C:STORE routines:ossl_store_handle_load_result:unsupported:crypto/store/store_result.c:151:
Could not open file or uri for loading certificate from data/badssl/certificates/tls-v1-0.badssl.com_1010_signed-public-key.pem: No such file or directory


Acquiring Certificate: https://tls-v1-1.badssl.com:1011


Could not find certificate from <stdin>
408FAAEC01000000:error:1608010C:STORE routines:ossl_store_handle_load_result:unsupported:crypto/store/store_result.c:151:
Could not open file or uri for loading certificate from data/badssl/certificates/tls-v1-1.badssl.com_1011_signed-public-key.pem: No such file or directory


Acquiring Certificate: https://tls-v1-2.badssl.com:1012
Acquiring Certificate: https://untrusted-root.badssl.com:443
Acquiring Certificate: https://upgrade.badssl.com:443
Acquiring Certificate: https://very.badssl.com:443
Acquiring Certificate: https://webpack-dev-server.badssl.com:443
Acquiring Certificate: https://wrong.host.badssl.com:443
Acquiring Certificate: https://www.badssl.com:443


## Display Basic Certificate Information

Displays select fields from each certificate.
- Subject
- Issuer
- Serial Number
- Validity Period

In [24]:
%%bash
FOLDER="data/badssl/certificates"

echo "Basic Certificate Details:"
for CERTIFICATE_PUBLIC_KEY_PEM in "${FOLDER}"/*_signed-public-key.pem; do
    echo "--------------------------------------------"
    DOMAIN=$(basename "${CERTIFICATE_PUBLIC_KEY_PEM}" | awk -F '_' '{print $1}')
    PORT=$(basename "${CERTIFICATE_PUBLIC_KEY_PEM}" | awk -F '_' '{print $2}')
    echo "Domain: ${DOMAIN}:${PORT}"
    openssl x509 -noout -in "${CERTIFICATE_PUBLIC_KEY_PEM}" \
        -subject -issuer -serial -startdate -enddate
done
echo ""


Basic Certificate Details:
--------------------------------------------
Domain: 1000-sans.badssl.com:443
subject=C=US, ST=California, L=Walnut Creek, O=Lucas Garron Torres, CN=1000-sans.badssl.com
issuer=C=US, O=DigiCert Inc, CN=DigiCert SHA2 Secure Server CA
serial=024C541761B4EB0CC512EEC67D1850DF
notBefore=Oct  5 00:00:00 2019 GMT
notAfter=Oct 13 12:00:00 2021 GMT
--------------------------------------------
Domain: badssl.com:443
subject=CN=*.badssl.com
issuer=C=US, O=Let's Encrypt, CN=R11
serial=03569BEE34CDE3271A5280D428FC00FF439B
notBefore=Aug  9 15:05:44 2024 GMT
notAfter=Nov  7 15:05:43 2024 GMT
--------------------------------------------
Domain: captive-portal.badssl.com:443
subject=C=US, ST=California, L=Walnut Creek, O=Lucas Garron Torres, CN=login.captive-portal.badssl.com
issuer=C=US, O=DigiCert Inc, CN=DigiCert SHA2 Secure Server CA
serial=052DA655C9CBC7D5DDD57B8410A3F428
notBefore=Jan 21 00:00:00 2020 GMT
notAfter=Jan 27 12:00:00 2022 GMT
-------------------------------

## Extract SubjectPublicKeyInfo

Process each certificate and extract the SubjectPublicKeyInfo field from the certificate file.

In [5]:
%%bash
FOLDER="data/badssl/certificates"

for CERTIFICATE_PEM in "${FOLDER}"/*_signed-public-key.pem; do
    # strip the path and suffix from the file name.
    TARGET=$(echo "${CERTIFICATE_PEM}" | sed 's/.*\///g' | sed -E 's/\_signed-public-key.(der|pem)$//g')
    
    PUBLIC_KEY_INFO_PEM="${FOLDER}/${TARGET}_public-key-info.pem"
    PUBLIC_KEY_INFO_DER="${FOLDER}/${TARGET}_public-key-info.der"
    echo "---"
    echo "Extracting SubjectPublicKeyInfo: ${CERTIFICATE_PEM}"

    echo "                     Saving PEM: ${PUBLIC_KEY_INFO_PEM}"
    openssl x509 -pubkey -inform PEM -in "${CERTIFICATE_PEM}" | \
        sed -n '/BEGIN\ PUBLIC\ KEY/,/END\ PUBLIC\ KEY/p' > "${PUBLIC_KEY_INFO_PEM}"

    echo "          Converting PEM to DER: ${PUBLIC_KEY_INFO_DER}"
    openssl asn1parse -inform PEM -in "${PUBLIC_KEY_INFO_PEM}" -out "${PUBLIC_KEY_INFO_DER}"
done
echo ""

---
Extracting SubjectPublicKeyInfo: data/badssl/certificates/1000-sans.badssl.com_443_signed-public-key.pem
                     Saving PEM: data/badssl/certificates/1000-sans.badssl.com_443_public-key-info.pem
          Converting PEM to DER: data/badssl/certificates/1000-sans.badssl.com_443_public-key-info.der
    0:d=0  hl=4 l= 290 cons: SEQUENCE          
    4:d=1  hl=2 l=  13 cons: SEQUENCE          
    6:d=2  hl=2 l=   9 prim: OBJECT            :rsaEncryption
   17:d=2  hl=2 l=   0 prim: NULL              
   19:d=1  hl=4 l= 271 prim: BIT STRING        
---
Extracting SubjectPublicKeyInfo: data/badssl/certificates/captive-portal.badssl.com_443_signed-public-key.pem
                     Saving PEM: data/badssl/certificates/captive-portal.badssl.com_443_public-key-info.pem
          Converting PEM to DER: data/badssl/certificates/captive-portal.badssl.com_443_public-key-info.der
    0:d=0  hl=4 l= 290 cons: SEQUENCE          
    4:d=1  hl=2 l=  13 cons: SEQUENCE          
    6

## Count of Thumbprint Types

Basic metrics on different thumbprint types.  _Certificate thumbprints_ represents the message digest of the binary (DER) certificate file.  _SPKI thumbprints_ represnets the message digest of the certificate's SubjectPublicKeyInfo field.

In [7]:
%%bash
FOLDER="data/badssl/certificates"

COUNT_CERTIFICATES=$(ls "${FOLDER}"/*signed-public-key.der | wc -l | awk '{print $1}')

UNIQUE_DER_DIGEST_COUNT=$(sha256sum "${FOLDER}"/*signed-public-key*.der \
    | awk '{print $1}' | sort | uniq | wc -l | awk '{print $1}')

UNIQUE_SPKI_DIGEST_COUNT=$(sha256sum "${FOLDER}"/*public-key-info.der \
    | awk '{print $1}' | sort | uniq | wc -l | awk '{print $1}')


echo "Summary:"
echo "  Certificates: ${COUNT_CERTIFICATES}"
echo "Thumbprints:"
echo "  Certificates: ${UNIQUE_DER_DIGEST_COUNT}"
echo "          SPKI: ${UNIQUE_SPKI_DIGEST_COUNT}"
echo ""

echo "Certificate (DER) Digests (SHA256):"
sha256sum "${FOLDER}"/*signed-public-key.der | sort
echo ""

echo "SubjectPublicKeyInfo Digests (SHA256):"
echo "[SubjectPublicKeyInfo derived from the Private Key]"
sha256sum "${FOLDER}"/*public-key-info.der | sort


Summary:
  Certificates: 66
Thumbprints:
  Certificates: 29
          SPKI: 9

Certificate (DER) Digests (SHA256):
020f82f2b8e8e5ab7407184708806cacec065f5063db54cf617659587374c6d0  data/badssl/certificates/sha1-2016.badssl.com_443_signed-public-key.der
069d03ce164013490ad78a069c0a37073cd03d7e710687a2d8f78884825a4262  data/badssl/certificates/untrusted-root.badssl.com_443_signed-public-key.der
0c105e23ec21fbc99371e4ed05b2b9cf4dc67aa06b5ab61d61502b50ed667039  data/badssl/certificates/edellroot.badssl.com_443_signed-public-key.der
1293933a9708f6529f82ad8fcccbf4f3ae939bbaecd42b2967d65ccbfc54601c  data/badssl/certificates/extended-validation.badssl.com_443_signed-public-key.der
141758610bd4d119c58e945674c6db2bd85e404bd6197af2ec8b80b8587268ab  data/badssl/certificates/sha1-2017.badssl.com_443_signed-public-key.der
181af79ec9d9fc2c62b1af332cf4bc67f487fdaa3dd118506609d521ff8387c6  data/badssl/certificates/no-sct.badssl.com_443_signed-public-key.der
1b20eeea08ca785164aa4077648feea9da01a81fb0706

In [8]:
%%bash
BASE_FOLDER="data/badssl"
CERT_FOLDER="${BASE_FOLDER}/certificates"
CSV="${BASE_FOLDER}"/certificate_inventory.csv
D="\t" 

echo -e "Target${D}Subject${D}SerialNumber${D}StartDate${D}EndDate${D}Thumbprint (SHA256)${D}SPKI (SHA256)" > "${CSV}"

for CERTIFICATE_DER in  "${CERT_FOLDER}"/*_signed-public-key.der; do
    # strip the path and suffix from the file name.
    TARGET=$(echo "${CERTIFICATE_DER}" | sed 's/.*\///g' | sed -E 's/\_signed-public-key.(der|pem)$//g')
    SUBJECT=$(openssl x509 -subject -noout -inform DER -in "${CERTIFICATE_DER}" | awk -F 'subject=' '{print $2}')
    SERIAL_NUMBER=$(openssl x509 -serial -noout -inform DER -in "${CERTIFICATE_DER}" | awk -F '=' '{print $2}')
    START_DATE=$(openssl x509 -startdate -noout -inform DER -in "${CERTIFICATE_DER}" | awk -F '=' '{print $2}')
    END_DATE=$(openssl x509 -enddate -noout -inform DER -in "${CERTIFICATE_DER}" | awk -F '=' '{print $2}')
    CERT_THUMBPRINT=$(sha256sum "${CERTIFICATE_DER}" | awk -F ' ' '{print $1}')
    SPKI_THUMBPRINT=$(sha256sum "${CERT_FOLDER}"/"${TARGET}"_public-key-info.der | awk '{print $1}')

    echo "--------------------------------"
    echo "      Target: ${TARGET}" && echo -ne "${TARGET}${D}" >> "${CSV}"
    echo "     Subject: ${SUBJECT}" && echo -ne "${SUBJECT}${D}" >> "${CSV}"
    echo "SerialNumber: ${SERIAL_NUMBER}" && echo -ne "${SERIAL_NUMBER}${D}" >> "${CSV}"
    echo "   StartDate: ${START_DATE}" && echo -ne "${START_DATE}${D}" >> "${CSV}"
    echo "     EndDate: ${END_DATE}" && echo -ne "${END_DATE}${D}" >> "${CSV}"
    echo " Cert Thumbprint: ${CERT_THUMBPRINT}"  && echo -ne "${CERT_THUMBPRINT}${D}" >> "${CSV}"
    echo " SPKI Thumbprint: ${SPKI_THUMBPRINT}" && echo -ne "${SPKI_THUMBPRINT}" >> "${CSV}"
    echo "" >> "${CSV}"

done

--------------------------------
      Target: 1000-sans.badssl.com_443
     Subject: C=US, ST=California, L=Walnut Creek, O=Lucas Garron Torres, CN=1000-sans.badssl.com
SerialNumber: 024C541761B4EB0CC512EEC67D1850DF
   StartDate: Oct  5 00:00:00 2019 GMT
     EndDate: Oct 13 12:00:00 2021 GMT
 Cert Thumbprint: ea0c6b7bc63e3ae5429fa19c09a070b65bcd198fde2d04c8b1ee261ce4b2bb0d
 SPKI Thumbprint: f522e496c72fccc623f1ffb9da5a79cdefe16340851f22d23d0cd2a58608066f
--------------------------------
      Target: captive-portal.badssl.com_443
     Subject: C=US, ST=California, L=Walnut Creek, O=Lucas Garron Torres, CN=login.captive-portal.badssl.com
SerialNumber: 052DA655C9CBC7D5DDD57B8410A3F428
   StartDate: Jan 21 00:00:00 2020 GMT
     EndDate: Jan 27 12:00:00 2022 GMT
 Cert Thumbprint: abdfbadb8f3264797f8279917f57d52cfbc7eb6e6725e0be9244f41788ae25c1
 SPKI Thumbprint: 7e364f1dec041d3acc0d7dc8d5e70421ea32dd61711f21a994e2efdbc9086ed2
--------------------------------
      Target: cbc.badssl.com_

## Remove Stored Files

In [9]:
%%bash

rm -rf data/badssl/*.*
rm -rf data/badssl/certificates